In [1]:
import os
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit
from pyspark.sql.functions import udf
from pyspark.sql.types import *
datasets_path = os.path.join('..','movie_recommendation_system', 'datasets')
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
tags_file = os.path.join(datasets_path, 'ml-latest', 'tags.csv')
links_file = os.path.join(datasets_path, 'ml-latest','links.csv')
movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')

In [2]:
pyspark

<module 'pyspark' from '/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/__init__.py'>

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
"""
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "50g") \
    .config("spark.submit.deployMode", "local") \
    .config("spark.executor.memory", "200g") \
    .config('spark.driver.maxResultSize', '200G') \
    .config('spark.executor.cores', '5')\
    .config('spark.memory.fraction ', '0')\
    .config('spark.memory.offHeap.enabled', 'true')\
    .config('spark.memory.offHeap.size', '2g')\
    .appName('movie-recomendation') \
    .getOrCreate()
"""

'\nspark = SparkSession.builder     .master(\'local[*]\')     .config("spark.driver.memory", "50g")     .config("spark.submit.deployMode", "local")     .config("spark.executor.memory", "200g")     .config(\'spark.driver.maxResultSize\', \'200G\')     .config(\'spark.executor.cores\', \'5\')    .config(\'spark.memory.fraction \', \'0\')    .config(\'spark.memory.offHeap.enabled\', \'true\')    .config(\'spark.memory.offHeap.size\', \'2g\')    .appName(\'movie-recomendation\')     .getOrCreate()\n'

In [4]:
sc.getConf().getAll()

[('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1618955806374'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '52160'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'lawn-128-61-44-86.lawn.gatech.edu'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

In [5]:
# conf1 = pyspark.SparkConf().setAll([('spark.executor.memory', '50g'), ('spark.executor.cores', '5'), ('spark.cores.max', '5'), ('spark.driver.memory','50g'), ('spark.memory.fraction', '0.01')])
# sc.stop()
# conf1 = pyspark.SparkConf().set("spark.jars", "/spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar")
# sc = spark.sparkContext
# sc = pyspark.SparkContext(conf=conf1).getOrCreate()
# spark = SparkSession.builder.config(conf=conf1).getOrCreate()
# sc.getConf().getAll()
# .config("spark.jars", "spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT.jar")

#spark = SparkSession.builder.appName("Spark_Processor").config("spark.jars", "spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT.jar").getOrCreate()
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("Spark_Processor").config("spark.redis.port", "6379").config("spark.jars", "spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar").config("spark.executor.memory", MAX_MEMORY).config("spark.driver.memory", MAX_MEMORY).config("spark.memory.fraction", "0.6").getOrCreate()
conf = pyspark.SparkConf()
sc = spark.sparkContext
sc.getConf().getAll()
#conf = pyspark.SparkConf()
#sc.getConf().getAll()

[('spark.jars',
  'spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar'),
 ('spark.app.id', 'local-1618955806374'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.memory', '5g'),
 ('spark.redis.port', '6379'),
 ('spark.driver.host', 'lawn-128-61-44-86.lawn.gatech.edu'),
 ('spark.driver.memory', '5g'),
 ('spark.app.name', 'Spark_Processor'),
 ('spark.rdd.compress', 'True'),
 ('spark.memory.fraction', '0.6'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.driver.port', '52160'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

In [6]:
print(sc.getConf().get('spark.driver.memory'))

5g


In [7]:
#Schema design which are as same as csv
ratingschema = StructType()\
    .add("userId", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("rating", DoubleType(), True)\
    .add("timeStamp", IntegerType(), True)
tagsschema = StructType()\
    .add("userId", IntegerType(), True)\
    .add("movieId", IntegerType(), True)\
    .add("tag", StringType(), True)\
    .add("timeStamp", IntegerType(), True)
linkschema = StructType()\
    .add("movieId", IntegerType(), True)\
    .add("IMDBID", IntegerType(), True)\
    .add("TMDBID", IntegerType(), True)

In [8]:
ratingdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(ratingschema)\
    .load(complete_ratings_file)
tagdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(tagsschema)\
    .load(tags_file)
linkdf = spark.read.format("csv")\
    .option("header",True)\
    .schema(linkschema)\
    .load(links_file)

In [8]:
sc.getConf().getAll()

[('spark.driver.port', '59182'),
 ('spark.app.name', 'Spark_Processor'),
 ('spark.app.id', 'local-1618867375618'),
 ('spark.jars',
  'spark-redis-branch-2.4/target/spark-redis_2.11-2.5.0-SNAPSHOT-jar-with-dependencies.jar'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'lawn-128-61-44-206.lawn.gatech.edu'),
 ('spark.redis.port', '6379'),
 ('spark.ui.showConsoleProgress', 'true')]

In [9]:
ratingdf.show(), tagdf, linkdf

+------+-------+------+----------+
|userId|movieId|rating| timeStamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



(None,
 DataFrame[userId: int, movieId: int, tag: string, timeStamp: int],
 DataFrame[movieId: int, IMDBID: int, TMDBID: int])

In [10]:
movie = spark.read.option("inferSchema", "true")\
    .option("header", "true").csv(movies_file)
movie

DataFrame[movieId: int, title: string, genres: string]

In [11]:
moviedf = movie.drop("genres")
genredf = movie.drop("title")
userdf = ratingdf.drop("movieId", "rating", "timeStamp")
userdf = userdf.select('userId').distinct()

In [12]:
moviedf, genredf, userdf

(DataFrame[movieId: int, title: string],
 DataFrame[movieId: int, genres: string],
 DataFrame[userId: int])

In [13]:
def ratingKey(user):
    out = "UserID: {},MovieID:".format(user)
    return out
udfRatingKey = udf(ratingKey, StringType())

In [12]:
moviedf = moviedf.select("title", "movieId")
# moviedf.show()

ratingdf = ratingdf.drop("timestamp")
ratingdf = ratingdf.withColumn("key", (concat(lit("User ID: "),col("userId"),lit(", Movie ID: "),col("movieID"))))
ratingdf = ratingdf.select("key", "userId", "movieId", "rating")
# ratingdf.show()

likedMoviedf = ratingdf.filter(ratingdf.rating > 4.0)
# likedMoviedf.show()

In [14]:
ratingdf.count()

25000095

In [15]:
#newMoviedf, otherVals = moviedf.randomSplit(weights = [0.10, 0.90])
import pyspark.sql.functions as f
my_list = newMoviedf.select(f.collect_list('movieId')).first()[0]
print(my_list[:10])

[208297, 174029, 190861, 189903, 146295, 97757, 155397, 779, 3420, 128878]


In [19]:
print(len(my_list))

6288


In [16]:
newRatingdf = ratingdf.filter(ratingdf['movieId'].isin(my_list))
newRatingdf.show()

+--------------------+------+-------+------+
|                 key|userId|movieId|rating|
+--------------------+------+-------+------+
|User ID: 1, Movie...|     1|   3569|   5.0|
|User ID: 1, Movie...|     1|   4308|   3.0|
|User ID: 1, Movie...|     1|   4325|   5.0|
|User ID: 1, Movie...|     1|   6377|   4.0|
|User ID: 1, Movie...|     1|   7318|   2.0|
|User ID: 1, Movie...|     1|   8405|   3.5|
|User ID: 2, Movie...|     2|   1201|   2.5|
|User ID: 2, Movie...|     2|   1270|   4.0|
|User ID: 2, Movie...|     2|   1275|   4.5|
|User ID: 2, Movie...|     2|   1287|   5.0|
|User ID: 2, Movie...|     2|   1693|   4.0|
|User ID: 2, Movie...|     2|   3098|   3.0|
|User ID: 2, Movie...|     2|   3510|   4.0|
|User ID: 2, Movie...|     2|   4571|   1.5|
|User ID: 2, Movie...|     2|   6156|   2.0|
|User ID: 2, Movie...|     2|   7162|   0.5|
|User ID: 2, Movie...|     2|   8010|   5.0|
|User ID: 2, Movie...|     2|   8961|   4.5|
|User ID: 2, Movie...|     2|  30848|   0.5|
|User ID: 

In [22]:
newLikedMoviedf = likedMoviedf.filter(likedMoviedf['movieId'].isin(my_list))
newLikedMoviedf.show()

+--------------------+------+-------+------+
|                 key|userId|movieId|rating|
+--------------------+------+-------+------+
|User ID: 1, Movie...|     1|   3569|   5.0|
|User ID: 1, Movie...|     1|   4325|   5.0|
|User ID: 2, Movie...|     2|   1275|   4.5|
|User ID: 2, Movie...|     2|   1287|   5.0|
|User ID: 2, Movie...|     2|   8010|   5.0|
|User ID: 2, Movie...|     2|   8961|   4.5|
|User ID: 3, Movie...|     3|     50|   5.0|
|User ID: 3, Movie...|     3|   1252|   5.0|
|User ID: 3, Movie...|     3|   2542|   4.5|
|User ID: 3, Movie...|     3|   4027|   4.5|
|User ID: 3, Movie...|     3|  56782|   4.5|
|User ID: 3, Movie...|     3|  71057|   4.5|
|User ID: 3, Movie...|     3| 136449|   5.0|
|User ID: 4, Movie...|     4|   8961|   5.0|
|User ID: 4, Movie...|     4|  48780|   5.0|
|User ID: 4, Movie...|     4|  71057|   5.0|
|User ID: 4, Movie...|     4| 171763|   4.5|
|User ID: 5, Movie...|     5|     50|   5.0|
|User ID: 5, Movie...|     5|   1407|   5.0|
|User ID: 

In [23]:
newMoviedf.write.format("org.apache.spark.sql.redis").option("table", "movie").option("key.column", "title").save()

In [24]:
newRatingdf.write.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").save()

In [25]:
newLikedMoviedf.write.format("org.apache.spark.sql.redis").option("table", "likedMovie").option("key.column", "key").save()

In [13]:
#read_moviedf = spark.read.format("org.apache.spark.sql.redis").option("table", "movie").option("key.column", "title").load()
#newMoviedf = read_moviedf.sort("title")
#newMoviedf.show()

#read_ratingdf = spark.read.format("org.apache.spark.sql.redis").option("keys.pattern", "rating:*").option("key.column", "key").load()
ratingdf = spark.read.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").load()
ratingdf.printSchema()
ratingdf.show()
#print(type(read_ratingdf))
#read_ratingdf.show()

#loadedDf = spark.read.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").load()
#loadedDf.printSchema()
#loadedDf.show()

#read_likedMoviedf = spark.read.format("org.apache.spark.sql.redis").option("table", "rating").option("key.column", "key").load()
#read_moviedf.printSchema()
#read_likedMoviedf.show()
#read_likedMoviedf

root
 |-- key: string (nullable = true)
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)

+--------------------+------+-------+------+
|                 key|userId|movieId|rating|
+--------------------+------+-------+------+
|User ID: 33836, M...| 33836|  26903|   5.0|
|User ID: 61047, M...| 61047|   6377|   4.0|
|User ID: 162132, ...|162132|  51709|   3.0|
|User ID: 46892, M...| 46892|   1270|   4.0|
|User ID: 65446, M...| 65446|  55290|   4.0|
|User ID: 9863, Mo...|  9863|    919|   4.5|
|User ID: 76678, M...| 76678|    419|   1.0|
|User ID: 29407, M...| 29407|     50|   4.5|
|User ID: 55564, M...| 55564|  48780|   3.0|
|User ID: 34060, M...| 34060|    551|   3.5|
|User ID: 12274, M...| 12274|   1641|   4.0|
|User ID: 36085, M...| 36085|   1208|   4.5|
|User ID: 144197, ...|144197|   4975|   3.0|
|User ID: 107650, ...|107650|  71057|   2.5|
|User ID: 147287, ...|147287|  64716|   5.0|
|User ID: 39592, M...| 39592|   

In [14]:
ratingdf.count()

--- Logging error ---
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/protocol

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/protocol

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/protocol

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/protocol

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/protocol

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/protocol

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/protocol

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:52158)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-e4a0975f122a>", line 1, in <module>
    ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/protocol

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:52158)

In [7]:
ratingdf, extraStuff = read_ratingdf.randomSplit([0.10, 0.90])
extraStuff.unpersist()

+--------------------+------+-------+------+
|                 key|userId|movieId|rating|
+--------------------+------+-------+------+
|User ID: 33836, M...| 33836|  26903|   5.0|
|User ID: 61047, M...| 61047|   6377|   4.0|
|User ID: 162132, ...|162132|  51709|   3.0|
|User ID: 46892, M...| 46892|   1270|   4.0|
|User ID: 65446, M...| 65446|  55290|   4.0|
|User ID: 9863, Mo...|  9863|    919|   4.5|
|User ID: 76678, M...| 76678|    419|   1.0|
|User ID: 29407, M...| 29407|     50|   4.5|
|User ID: 55564, M...| 55564|  48780|   3.0|
|User ID: 12274, M...| 12274|   1641|   4.0|
|User ID: 34060, M...| 34060|    551|   3.5|
|User ID: 36085, M...| 36085|   1208|   4.5|
|User ID: 144197, ...|144197|   4975|   3.0|
|User ID: 107650, ...|107650|  71057|   2.5|
|User ID: 147287, ...|147287|  64716|   5.0|
|User ID: 39592, M...| 39592|   5478|   2.5|
|User ID: 122955, ...|122955|   1892|   2.0|
|User ID: 60445, M...| 60445|  52460|   4.5|
|User ID: 88349, M...| 88349|   1921|   4.0|
|User ID: 

In [7]:
read_ratingdf.count()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:60822)
Traceback (most recent call last):
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-edde5b0c58a7>", line 1, in <module>
    read_ratingdf.count()
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/rbanerjee/opt/anaconda3/envs/moviesys/lib/python3.6/site-packages/py4j/prot

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:60822)

we need to connect Redis here

# ALS: collaborative filtering for explicit data

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [15]:
#ratingforals = read_ratingdf
#ratingforals.show()
ratingforals = ratingdf
ratingforals.show()
#spark.driver.memoryOverhead
#spark.executor.memoryOverhead

+--------------------+------+-------+------+
|                 key|userId|movieId|rating|
+--------------------+------+-------+------+
|User ID: 1, Movie...|     1|    296|   5.0|
|User ID: 1, Movie...|     1|    306|   3.5|
|User ID: 1, Movie...|     1|    307|   5.0|
|User ID: 1, Movie...|     1|    665|   5.0|
|User ID: 1, Movie...|     1|    899|   3.5|
|User ID: 1, Movie...|     1|   1088|   4.0|
|User ID: 1, Movie...|     1|   1175|   3.5|
|User ID: 1, Movie...|     1|   1217|   3.5|
|User ID: 1, Movie...|     1|   1237|   5.0|
|User ID: 1, Movie...|     1|   1250|   4.0|
|User ID: 1, Movie...|     1|   1260|   3.5|
|User ID: 1, Movie...|     1|   1653|   4.0|
|User ID: 1, Movie...|     1|   2011|   2.5|
|User ID: 1, Movie...|     1|   2012|   2.5|
|User ID: 1, Movie...|     1|   2068|   2.5|
|User ID: 1, Movie...|     1|   2161|   3.5|
|User ID: 1, Movie...|     1|   2351|   4.5|
|User ID: 1, Movie...|     1|   2573|   4.0|
|User ID: 1, Movie...|     1|   2632|   5.0|
|User ID: 

In [9]:
stuffToUse, stuffNotToUse = ratingforals.randomSplit([0.10, 0.90])

In [10]:
stuffNotToUse.unpersist(True)

DataFrame[key: string, userId: int, movieId: int, rating: double]

In [16]:
(training, test) = ratingforals.randomSplit([0.5, 0.5])
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# c
als = ALS(maxIter=3, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", 
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
#movieRecs = model.recommendForAllItems(10)

Root-mean-square error = 0.9172555182587768


error message: 
1. when executor = 50g, 200g: Job aborted due to stage failure: Task 1 in stage 7.0 failed 1 times, most recent failure: Lost task 1.0 in stage 7.0 (TID 22, 192.168.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space
2. additional maxreusltsize = 20G: Py4JJavaError: An error occurred while calling o145.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 7.0 failed 1 times, most recent failure: Lost task 6.0 in stage 7.0 (TID 27, 192.16An error occurred while calling o381.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 21, 192.168.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space8.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space\
maybe this:
https://knowledge.informatica.com/s/article/567632?language=en_US\
An error occurred while calling o381.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 21, 192.168.1.73, executor driver): java.lang.OutOfMemoryError: Java heap space

In [19]:
userRecs.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [20]:
print(type(ratingforals))

<class 'pyspark.sql.dataframe.DataFrame'>


In [29]:
userRecs.show(3)

KeyboardInterrupt: 

In [28]:
users = training.select(als.getUserCol()).distinct().limit(3)
users.show()

+------+
|userId|
+------+
| 10206|
| 10362|
| 10623|
+------+



In [31]:
userSubsetRecs = model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
| 10362|[[194268, 21.7390...|
| 10623|[[202231, 13.1945...|
| 10206|[[128091, 29.2722...|
+------+--------------------+



In [32]:
userRecs.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [40]:
userRecs.select("recommendations").show(3)

KeyboardInterrupt: 

In [42]:
userRecs.select("userId")

DataFrame[userId: int]

In [44]:
from pyspark.sql.functions import col
userSubsetRecs.filter(col("userId") == 10362) \
    .show(truncate=False) 

+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                                                 |
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|10362 |[[194268, 21.739002], [173947, 21.160053], [185665, 20.949986], [202231, 20.26989], [174105, 20.036861], [170509, 18.978056], [181155, 18.821554], [205453, 18.25892], [126552, 18.156422], [124735, 18.065588]]|
+------+------------------------------------------------------------------------------------------------------------------------

In [45]:
result = userRecs.filter(userRecs.userId == 10362).collect()
row = result[0]

In [46]:
row.asDict().values()

dict_values([10362, [Row(movieId=194268, rating=21.739002227783203), Row(movieId=173947, rating=21.160053253173828), Row(movieId=185665, rating=20.94998550415039), Row(movieId=202231, rating=20.26988983154297), Row(movieId=174105, rating=20.036861419677734), Row(movieId=170509, rating=18.978055953979492), Row(movieId=181155, rating=18.82155418395996), Row(movieId=205453, rating=18.258920669555664), Row(movieId=126552, rating=18.156421661376953), Row(movieId=124735, rating=18.065587997436523)]])

In [59]:
item = row.asDict()["recommendations"]
idList = []
for x in item:
    idList.append(x.asDict()["movieId"])
print(idList)

[194268, 173947, 185665, 202231, 174105, 170509, 181155, 205453, 126552, 124735]


In [58]:
moviedf.printSchema()

root
 |-- title: string (nullable = true)
 |-- movieId: integer (nullable = true)



In [61]:
moviedf.where(moviedf["movieId"].isin(idList)).show(truncate = False)

+---------------------------------------------------------------------------+-------+
|title                                                                      |movieId|
+---------------------------------------------------------------------------+-------+
|Love Bites (1993)                                                          |124735 |
|Fort McCoy (2011)                                                          |126552 |
|Esteros (2016)                                                             |170509 |
|The Great Piggy Bank Robbery (1946)                                        |173947 |
|Grandma's Reading Glass (1900)                                             |174105 |
|Gilbert (2017)                                                             |181155 |
|WWE: You Think You Know Me? The Story of Edge (2012)                       |185665 |
|Farming (2018)                                                             |194268 |
|Foster (2018)                                        

In [65]:
from pyspark.sql.functions import col, when

moviedf.where(moviedf["movieId"].isin(idList)).orderBy(when(col("movieId") == idList[0], 1)
           .when(col("movieId") == idList[1], 2)
           .when(col("movieId") == idList[2], 3)
           .when(col("movieId") == idList[3], 4)
           .when(col("movieId") == idList[4], 5)
           .when(col("movieId") == idList[5], 6)
           .when(col("movieId") == idList[6], 7)
           .when(col("movieId") == idList[7], 8)
           .when(col("movieId") == idList[8], 9)
           .when(col("movieId") == idList[9], 10)
           ).show(truncate = False)

+---------------------------------------------------------------------------+-------+
|title                                                                      |movieId|
+---------------------------------------------------------------------------+-------+
|Farming (2018)                                                             |194268 |
|The Great Piggy Bank Robbery (1946)                                        |173947 |
|WWE: You Think You Know Me? The Story of Edge (2012)                       |185665 |
|Foster (2018)                                                              |202231 |
|Grandma's Reading Glass (1900)                                             |174105 |
|Esteros (2016)                                                             |170509 |
|Gilbert (2017)                                                             |181155 |
|The Good Fight: The Abraham Lincoln Brigade in the Spanish Civil War (1984)|205453 |
|Fort McCoy (2011)                                    

In [66]:
sc.stop()

In [67]:
exit()